In [1]:
#Importamos todas las librerias que vamos a utilizar en este projecto.
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
import seaborn as sns
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

#usamos el API de kaggle para descargarnos unos data sets con el precio historico de varias cryptomonedas
! kaggle datasets download -d sudalairajkumar/cryptocurrencypricehistory
!tar -xzvf cryptocurrencypricehistory.zip
!rm -rf cryptocurrencypricehistory.zip

#Creamos varios data frames con las monedas que vamos a trabajar.
df_bitcoin = pd.read_csv('coin_Bitcoin.csv', parse_dates = ['Date'])
df_ether = pd.read_csv('coin_Ethereum.csv', parse_dates = ['Date'])

#Creamos 3 dataframes con de las monedas que vamos a usar
ether = pd.DataFrame(df_ether)
bitcoin = pd.DataFrame(df_bitcoin)
ether.Date.dtypes

#WE CREATE A datetime column using the pd.to_datetime function
ether['datetime'] = pd.to_datetime(ether['Date'])
#we drop the date time column in string format
ether.drop('Date', axis=1, inplace = True)
#we do the same for bitcoin
bitcoin['datetime'] = pd.to_datetime(bitcoin['Date'])
bitcoin.drop('Date', axis = 1, inplace = True)

#definimos una funcion que convierte un valor timestamp a un valor datetime usando un apply
#la funcion necesita tener datetime importado
def ts_to_dt(df, column, new_column):
    df[str(new_column)] = df[column].apply(datetime.fromtimestamp)
    return df

# We define some functions that will help us create some columns we are interested in.
def crypto_prev_close (df):
    for v in range(1, len(df)):
        df.loc[v, 'previousClose'] = df.loc[v - 1, 'Close']
    return df
def crypto_return(df):
    df['percent_change'] = df['Close'].pct_change()
    return df

ether = crypto_prev_close(ether)
ether = crypto_return(ether)

bitcoin = crypto_prev_close(bitcoin)
bitcoin = crypto_return(bitcoin)

bitcoin.drop(['Volume', 'Marketcap', 'Open', 'SNo', 'Name'], axis =1, inplace = True)
ether.drop(['Volume', 'Marketcap', 'Open', 'SNo', 'Name'], axis =1, inplace = True)
#Creamos un df que combine Bitcoin and Etherum
bit_eth2 = bitcoin.merge(ether, how = 'left', on = 'datetime', suffixes = ('_bit', '_eth') )

#We grab the values for the age range we are interested in for our analysis
bit_eth = bit_eth2.loc[bit_eth2.loc[:, "datetime"] >= '2016-04-11']

#we extract a sub dataframe that we will use for our analysis
return_crypto = bit_eth[['percent_change_bit', 'percent_change_eth', 'datetime']]
return_crypto['datetime'] = return_crypto['datetime'].dt.date

#We set the index to datetime same as stock market data
return_crypto = return_crypto.set_index('datetime')

#We call an API for Stock Data
url_yahoo = "https://yahoo-finance-low-latency.p.rapidapi.com/v8/finance/spark"
parameters = {"symbols":"AAPL,MSFT,GME,^GSPC,","range":"5y","interval":"1d"}
headers = {
    'x-rapidapi-key': "5978a55496msh35be22b262fe8acp19f68cjsn3abe53806b27",
    'x-rapidapi-host': "yahoo-finance-low-latency.p.rapidapi.com"
    }
response = requests.request("GET", url_yahoo, headers=headers, params=parameters)

#We Create a DF for MSFT values
response1 = response.json()
MSFT = pd.DataFrame(response1['MSFT'])
# we get a new datetime column out of time stamp
msft = ts_to_dt(MSFT, 'timestamp', 'datetime')
#we create a dataframe and create a date time column for apple
AAPL = pd.DataFrame(response1['AAPL'])
aapl = ts_to_dt(AAPL, 'timestamp', 'datetime')
GME = pd.DataFrame(response1['GME'])
gme = ts_to_dt(GME, 'timestamp', 'datetime')
#We create a dataframe and date time column for the SP500 index
GSPC = pd.DataFrame(response1['^GSPC'])
gspc = ts_to_dt(GSPC, 'timestamp', 'datetime')

#We Notice that the data frame is missing values in the previous close column
#we create a function that calculates the value using the previous rows close value
def previous_close(df):
    for v in range(1, len(df)):
        df.loc[v, 'previousClose'] = df.loc[v - 1, 'close']
    return df

#apply to sp500
SP500 = previous_close(gspc)
# apply previous_close to apple
AAPL = previous_close(aapl)

# apply previous_close to msft
MSFT = previous_close(msft)
GME = previous_close(gme)

#drop unneed columns for 3 data sets
MSFT.drop(['end', 'start', 'timestamp', 'dataGranularity','chartPreviousClose'], axis =1, inplace = True)

#We drop the values that we don't need for this analysis
AAPL.drop(['end', 'start', 'timestamp', 'dataGranularity', 'chartPreviousClose'], axis =1, inplace = True)
SP500.drop(['end', 'start', 'timestamp', 'dataGranularity', 'chartPreviousClose'], axis =1, inplace = True)
GME.drop(['end', 'start', 'timestamp', 'dataGranularity', 'chartPreviousClose'], axis =1, inplace = True)

#We extract only the date in datetime
MSFT['datetime'] = MSFT['datetime'].dt.date
GME['datetime'] = GME['datetime'].dt.date
AAPL['datetime'] = AAPL['datetime'].dt.date
SP500['datetime'] = SP500['datetime'].dt.date

#We create a function to create a percentage change column by inputing a dataframe from the yahoo finance api
def percent_change(df):
    df['percent_change'] = df['close'].pct_change()
    return df
    
#apply percent_change to 3 data sets
aapl = percent_change(AAPL)
sp500 = percent_change(SP500)
msft = percent_change(MSFT)
gme = percent_change(GME)

# We create a new column that tracks the growth of the stock since the first day in our dataframe, 
#this alows us to bettter compare across a big range of prices
# this is the function for stocks:
def base_price(df):
    for v in range(0, len(df)):
        df.loc[v, 'price_index'] = df.loc[v, 'close'] / df.loc[0, 'close']
    return df

#We create the same function with different columns to iterate over our crypto data
def crypto_index(df):
    for v in range(0, len(df)):
        df.loc[v, 'price_index'] = df.loc[v, 'Close'] / df.loc[0, 'Close']
    return df

#We create a price index column for all stock dataframes 
gme2 = base_price(gme)
aapl2 = base_price(aapl)
msft2 = base_price(msft)
sp501 = base_price(sp500)
sp501.drop(columns = ['close', 'previousClose', 'percent_change'], axis =1)
gme2.drop(columns = ['close', 'previousClose', 'percent_change'], axis =1)
msft2.drop(columns = ['close', 'previousClose', 'percent_change'], axis =1)
aapl2.drop(columns = ['close', 'previousClose', 'percent_change'], axis =1)

#Set the index as the date column for our price index stocks
aapl_pi = aapl2.set_index('datetime')
msft_pi = msft2.set_index('datetime')
gme_pi = gme2.set_index('datetime')
sp500_pi = sp501.set_index('datetime')

# We merge all the price index into one data frame 
ms_aap_pi = msft_pi.merge(aapl_pi, how = 'left', on = 'datetime', suffixes = ('_msft', '_aapl') )
ms_aap_gme_pi = ms_aap_pi.merge(gme_pi, how = 'left', on = 'datetime', suffixes = ('_msft', '_aapl', '_gme'))
all_tog_pi = ms_aap_gme_pi.merge(sp500_pi, how = 'left', on = 'datetime', suffixes = ('_gme', '_sp500'))

#We create a new dataframe with only the date time and price index columns
price_index = all_tog_pi[['price_index_msft', 'price_index_aapl', 'price_index_gme', 'price_index_sp500']]

#We create crypto dataframes with a price index column 
ether = ether.loc[ether.loc[:, 'datetime'] >= '2016-04-11']
bitcoin = bitcoin.loc[bitcoin.loc[:, 'datetime'] >= '2016-04-11']
ether = ether.reset_index()
bitcoin = bitcoin.reset_index()

ether_indexed = crypto_index(ether)
bit_indexed = crypto_index(bitcoin)

# We merge the crypto indexed dataframes to one dataframe
crypto_indexed = bit_indexed.merge(ether_indexed, on = 'datetime', how = 'left', suffixes = ('_bit', '_ether'))
crypto_indexed = crypto_indexed[['price_index_bit', 'price_index_ether', 'datetime']]
crypto_indexed2 = crypto_indexed.loc[crypto_indexed.loc[:, 'datetime'] >= '2016-04-11']
crypto_indexed2.set_index('datetime', inplace = True)

#We plot a timeseries of price index for etherum and bitcoin
fig, ax = plt.subplots(figsize = (20, 15))
ax.set_facecolor('gray')
sns.lineplot(data = crypto_indexed2, x = 'datetime', y = 'price_index_ether', color = 'cyan', ax = ax, label ='ether')
sns.lineplot(data = crypto_indexed2, x = 'datetime', y = 'price_index_bit', color = 'yellow', ax = ax, label = 'bitcoin')
ax.set_ylabel('price index crypto')
ax.set_title('Crypto Price Index Time Series')
plt.legend()
plt.savefig("Crypto Price Index Over Time", dpi=550)
plt.show()

#We plot a timeseries for other stocks
fig, ax = plt.subplots(figsize = (20, 15))
ax.set_ylabel('price index stocks')
sns.lineplot(data = price_index, x = 'datetime', y = 'price_index_msft', color = 'red', ax = ax, label = 'msft')
sns.lineplot(data = price_index, x = 'datetime', y = 'price_index_gme', color = 'magenta', ax = ax, label = 'gme')
sns.lineplot(data = price_index, x = 'datetime', y = 'price_index_sp500', color = 'blue', ax = ax, label = 'sp500')
sns.lineplot(data = price_index, x = 'datetime', y = 'price_index_aapl', color = 'green', ax = ax, label = 'aapl')
plt.legend()
ax.set_title('Stocks Price Index Time Series')
plt.savefig("Stock Price Index Over Time", dpi=550)

#We start calculating the Sharpe Ratio

#Set the index as the date column for our stocks
msft = msft.set_index('datetime')
aapl = aapl.set_index('datetime')
sp500 = sp500.set_index('datetime')
gme = gme.set_index('datetime')

#Creating a Df of returns for our 2 stocks so that we can subtract the sp_500 returns
#this is done to calculate the sharpe ratio
ms_aap = msft.merge(aapl, how = 'left', on = 'datetime', suffixes = ('_msft', '_aapl') )

all_together = ms_aap.merge(sp500, how = 'left', on ='datetime', suffixes = ('_msft', '_aapl', '_sp500'))

all_together2 = all_together.merge(gme, how = 'left', on = 'datetime', suffixes = ('_sp500', '_gme'))

# We keep only the percent change column for the stocks we are interested in
msft_aapl_gme_returns = all_together2[['percent_change_msft', 'percent_change_aapl', 'percent_change_gme']]

msft_aapl_gme_returns

sp500_returns = all_together2['percent_change_sp500']

#we create a dataframe with the excess returns for each stock compared to the sp500. We also create a graph
excess1_returns = msft_aapl_gme_returns.sub(sp500_returns, axis = 0)
excess1_returns.plot(alpha = 0.2)
plt.savefig("Excess Returns Stocks", dpi=550)
Crypto_excess = return_crypto.sub(sp500_returns, axis = 0)
Crypto_excess
Crypto_excess.plot(alpha = 0.6)
plt.savefig("Excess Returns Crypto", dpi=550)

#Calculating the Sharpe Ratio 
#Step 1 Average Excess Return 
#AVG Excess for Crypto
avg_crypto_excess = Crypto_excess.mean()
avg_crypto_excess.plot.bar(title = 'Avg Excess Crypto Return over SP500', color = 'red')
plt.savefig("AVG Excess Returns Crypto", dpi=550)

#Avg Excess Return Stocks
avg_excess_returns = excess1_returns.mean()
avg_excess_returns.plot.bar(title = 'Avg of Excess Returns over SP500')
plt.savefig("AVG Excess Returns Stocks", dpi=550)

#Calculating Sharpe Ratio
#Step 2: Calculating Standard Deviation of Excess Returns
#STD for stocks
std_excess = excess1_returns.std()
std_excess.plot.bar(title = 'Std of Stocks Excess Returns over SP500')
#we can see that apple has a higher degree of volatility than microsoft. 
plt.savefig("Risk Excess Returns Stock", dpi=550)

#STD Crypto
crypto_std_excess = Crypto_excess.std()
crypto_std_excess.plot.bar(title = 'Std of Crypto Excess Returns over SP500', color = 'red')
plt.savefig("Risk Excess Returns Crypto", dpi=550)


#Sharpe Ratio - Calculating Daily & Anualized Sharpe Ratio (Ratio of avg excess returs / sd excess returns)
#SP500
daily_sharpe_ratio = avg_excess_returns.div(std_excess)
daily_sharpe_ratio.plot.bar(title = 'Daily Sharpe Ratio Stocks (SP500)')
annual_factor = np.sqrt(252)
annual_sharpe_ratio = daily_sharpe_ratio.mul(annual_factor)
annual_sharpe_ratio.plot.bar(title = 'Anualized Sharpe Stocks (SP500)')
plt.savefig("Anualized Sharpe Ratio Stocks", dpi=550)


#Sharpe Ratio - Calculating Daily & Anualized Sharpe Ratio for Crypto (Ratio of avg excess returs / sd excess returns)
daily_sharpe_ratio_cr = avg_crypto_excess.div(crypto_std_excess)
daily_sharpe_ratio_cr.plot.bar(title = 'Daily Sharpe Ratio Crypto')
annual_factor = np.sqrt(252)
annual_sharpe_ratio_cr = daily_sharpe_ratio_cr.mul(annual_factor)
annual_sharpe_ratio_cr.plot.bar(title = 'Anualized Sharpe Crypto (SP500)', color = 'red')
plt.savefig("Anualized Sharpe Ratio Crypto", dpi=550)


# We join all the data on annual sharpe ratios together into a nice graph. 
total_sharpe = {'asset' : ['bitcoin', 'etherum', 'microsoft', 'apple', 'gamestop'],
'annualized Sharpe Ratio' : [1.320, 1.151, 1.186, .994, .692] }
total_sharpes = pd.DataFrame(total_sharpe, columns = ['asset', 'annualized Sharpe Ratio'])
sns.barplot(data = total_sharpes, x= 'asset', y= 'annualized Sharpe Ratio')
plt.title('Anualized Sharpe Ratio All Assets')
plt.savefig("Annualized Sharpe Ratio All Assets", dpi=550)


#We plot the return for each stock across a datetime variable, and then plot the distribution of percent changes.
fig, axs = plt.subplots(nrows=6, ncols=2, figsize=(50, 45))
sns.lineplot(x = 'datetime', y = 'percent_change', data = msft, color = 'red', ax = axs[0,0])
sns.histplot(x = 'percent_change', data = msft, ax = axs[0,1], color = 'red')
sns.lineplot( x = 'datetime', y = 'percent_change', data = aapl, color = 'green', ax = axs [1, 0])
sns.histplot(x = 'percent_change', data = aapl, ax = axs[1,1], color = 'green')
sns.lineplot(x = 'datetime', y = 'percent_change', data = sp500, color = 'blue', ax = axs [2, 0])
sns.histplot(x = 'percent_change', data = sp500, ax = axs[2,1], color = 'blue')
sns.lineplot(x = 'datetime', y = 'percent_change', data = gme, color = 'magenta', ax = axs [3, 0])
sns.histplot(x = 'percent_change', data = gme, ax = axs[3,1], color = 'magenta')
sns.lineplot(x = 'datetime', y = 'percent_change', data = bitcoin, color = 'yellow', ax = axs [4, 0])
sns.histplot(x = 'percent_change', data = bitcoin, ax = axs[4,1], color = 'yellow')
sns.lineplot(x = 'datetime', y = 'percent_change', data = ether, color = 'cyan', ax = axs [5, 0])
sns.histplot(x = 'percent_change', data = ether, ax = axs[5,1], color = 'cyan')
axs[0,0].set_title("Percent Change MSFT")
axs[0,1].set_title("Percent Change Distribution MSFT")
axs[1,0].set_title("Percent Change AAPL")
axs[1,1].set_title("Percent Change Distribution AAPL")
axs[2,0].set_title("Percent Change SP500")
axs[2,1].set_title("Percent Change Distribution SP500")
axs[3,0].set_title("Percent Change GME")
axs[3,1].set_title("Percent Change Distribution GME")
axs[4,0].set_title("Percent Change Bitcoin")
axs[4,1].set_title("Percent Change Distribution Bitcoin")
axs[5,0].set_title("Percent Change Ether")
axs[5,1].set_title("Percent Change Distribution Ether")
plt.savefig("Percent Change and Distribution of Percentage Changes", dpi=550)